**First Use Examples**

In order to interact with a scenario, a Client first needs to initialized by
pointing to a specific scenario in the ETM. There are several options to establish
a connection, e.g. by creating a new scenario, creating a copy of an existing scenario 
or reconnecting to a previously created scenario.

Note that the parameter `scenario_id` refers to which is also known as an `api_session_id`. 
This id does not directly refers to a `saved_scenario_id` of scenarios stored under 
an ETM user account.

In [ ]:
from pyetm import Client

# create a new scenario from scratch
client = Client.from_scenario_parameters(end_year=2050, area_code="nl2019")

# print scenario_id
scenario_id = client.scenario_id
scenario_id

In [ ]:
from pyetm import Client

# you can reconnect to your scenario with the provided scenario_id
client = Client(scenario_id)
client.scenario_id

**Scenario properties**

After a connection with a scenario is established, you can request specific 
properties of that given scenario. The more basic properties that can be accessed 
for example are the start year, area code, or creation data. Some properties can 
also be changed, for example if the scenario is read only and thus cannot be modified 
via the API.

In [ ]:
client.created_at

In [ ]:
# frst check which parameters can be set in the scenario
parameters = client.input_parameters
parameters.iloc[41:46]

In [ ]:
# show parameters that are set by the user
client.get_input_parameters(user_only=True)

In [ ]:
# let's the capacities of several plants
# we can specify these changes in a dictonairy or series object.
user_values = {
    'capacity_of_energy_power_nuclear_gen2_uranium_oxide' : 1750,
    'capacity_of_energy_power_wind_turbine_coastal' : 50,
    'capacity_of_energy_power_wind_turbine_offshore' : 25000
}

# apply the changes to the scenario
client.input_parameters = user_values
client.get_input_parameters(user_only=True)

**Result Curves**

Simular to the scenario properties, the results of a scenario can also be accessed 
as properties of the scenario. When scenario parameters are changed, all result 
curves are automatically reset and are requested again upon accessing the client 
property. This means that the first time that a result curve is requested some time 
will pass before the result is loaded, as the ETM is evaluating the scenario in the 
background.

In [ ]:
# request hourly electricity curves
ecurves = client.hourly_electricity_curves
ecurves.head()

**Custom Curves**

It is also possible to upload custom curves for a select number of parameters.

In [ ]:
# show if there are custom curves attached in the scenario
client.get_custom_curve_settings()

Set some scenarios by passing random data for the interconnectors

In [ ]:
import numpy as np
import pandas as pd

# create prices for interconnectors
con1 = pd.Series(np.random.rand(8760)*3, name='interconnector_1_price')
con2 = pd.Series(np.random.rand(8760)*49, name='interconnector_2_price')
con3 = pd.Series(np.random.rand(8760)*5, name='interconnector_3_price')

# create capacity factors for wind
offshore = pd.Series(np.random.rand(8760), name='weather/wind_offshore_baseline')
onshore = pd.Series(np.random.rand(8760), name='weather/wind_inland_baseline')

# merge series in dataframe
ccurves = pd.concat([con1, con2, offshore, onshore], axis=1)

# show random data
ccurves.head()

In [ ]:
# set data as ccurves profiles
client.set_custom_curves(ccurves)

In [ ]:
# add additional custom curve
client.upload_custom_curves(con3)
client.custom_curves.head()

In [ ]:
# show parameters that have been overwritten by the custom curves
client.get_custom_curve_user_value_overrides()

In [ ]:
# show details about the uploaded ucstom curves
client.get_custom_curve_settings()